In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "a": np.random.random(size = size)
    , "b": np.random.random(size = size)
    , "x1" : range(size)
    , "x2" : range(size, size + size)
    , "y": range(-size, 0)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
})
df.head()

f,time_idx,dummy,a,b,x1,x2,y,actual,predicted,dummy_groups
f64,i64,str,f64,f64,i64,i64,i64,i32,f64,str
0.0,0,"""a""",0.344781,0.661369,0,10000,-10000,0,0.306428,"""a"""
0.841471,1,"""a""",0.197865,0.928313,1,10001,-9999,0,0.801933,"""a"""
0.909297,2,"""a""",0.948278,0.073713,2,10002,-9998,1,0.329489,"""a"""
0.14112,3,"""a""",0.605177,0.8063,3,10003,-9997,1,0.427147,"""a"""
-0.756802,4,"""a""",0.965519,0.636594,4,10004,-9996,0,0.387884,"""a"""


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.query_jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Convolution (by FFT). 
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full"),
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.344781,-0.661369
-0.841471,-0.197865,-0.928313
-0.909297,-0.948278,-0.073713
-0.14112,-0.605177,-0.8063
0.756802,-0.620738,0.024775


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

y
list[f64]
"[2.0, -1.0]"


In [8]:
df.select(
    pds.query_lstsq_report(
        # str | pl.Expr
        "x1", "x2",
        target = pl.col("y"),
        add_bias=False
    ).alias("report")
).unnest("report")

feat_idx,coeff,std_err,t,p>|t|
u16,f64,f64,f64,f64
0,2.0,2.3854e-16,8.3842e15,0.0
1,-1.0,9.0158e-17,-1.1092e16,0.0


In [9]:
df.lazy().select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = "y", # We can either put pl.col("y") here or just the string "y"
        add_bias=False
    )
).collect()

y
list[f64]
"[2.0, -1.0]"


In [10]:
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
).head() 

coeffs
list[f64]
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.query_lstsq(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
i64,i64,i64,f64,f64
0,10000,-10000,-10000.0,5.8208e-11
1,10001,-9999,-9999.0,5.8208e-11
2,10002,-9998,-9998.0,5.8208e-11
3,10003,-9997,-9997.0,5.8208e-11
4,10004,-9996,-9996.0,5.8208e-11


In [12]:
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""b""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"


In [13]:
# Rolling regression, kind of slow rn
df.lazy().rolling(
    index_column = pl.col("time_idx").set_sorted(),
    period = "30i",
    # offset = "-1i"
).agg(
    pds.query_lstsq(pl.col("x1"), pl.col("x2"), target = pl.col("y"), add_bias=False).alias("coefficients")
).slice(offset = 30).select(
    "time_idx",
    "coefficients",
).collect().head(10)

time_idx,coefficients
i64,list[f64]
30,"[2.0, -1.0]"
31,"[2.0, -1.0]"
32,"[2.0, -1.0]"
33,"[2.0, -1.0]"
34,"[2.0, -1.0]"
35,"[2.0, -1.0]"
36,"[2.0, -1.0]"
37,"[2.0, -1.0]"
38,"[2.0, -1.0]"


In [14]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


# ML Metrics

In [15]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""a""",0.327543,0.9847,0.501435,0.495142,0.249134,0.500682,0.51132
"""b""",0.33748,1.011861,0.491207,0.493178,0.246095,0.489005,0.489277


# Str Extension

In [16]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""


In [17]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""church"""
"""world"""
"""to"""
"""hello"""
"""going"""


In [18]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""hello"""
"""world"""
"""church"""
"""go"""
""""""


In [19]:
df2.select(
    pds.str_leven("word", "world")
).head()

word
u32
2
1
2
1
2


In [20]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", "world")
).head()

word
u32
2
1
2
1
2


In [21]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", "world", return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [22]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), "world", 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""


In [23]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [24]:
df.filter(
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [25]:
df.select(
    pds.str_leven("word", "asasasa", return_sim=True).alias("asasasa"),
    pds.str_leven("word", "sasaaasss", return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", "asdasadadfa", return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", "apples").alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", "apples", return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", "apples").alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [26]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
0.822892
-0.188963
-2.320337


In [27]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pl.col("a").stats.rand_normal(mean = 0.5, std = 1., respect_null=True).alias("random")
).head()

a,random
f64,f64
null,null
null,null
0.822892,0.038477
-0.188963,1.441658
-2.320337,0.514815


In [28]:
# Genenrate random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 1, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
0.822892,"""Qk"""
-0.188963,"""jHg6e"""
-2.320337,"""V"""


In [29]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 5, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
0.822892,"""JQehU"""
-0.188963,"""ojtr1"""
-2.320337,"""pT2YZ"""


In [30]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pl.col("a").stats.rand_normal().alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.col("a").stats.rand_uniform(low = 0., high = None, respect_null=True).alias("test2")
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,test2,test1_perturbed
f64,f64,f64,f64
null,-0.382368,null,-0.382395
null,0.943623,null,0.94338
0.822892,0.847817,0.329099,0.847651
-0.188963,0.614026,1.004053,0.614207
-2.320337,0.562385,2.255311,0.562026


In [31]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.746068,-1.340912,9
null,0.870044,-0.62221,3
0.822892,0.929766,0.521739,9
-0.188963,0.302698,1.556166,7
-2.320337,0.82819,2.398959,3


In [32]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.query_ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
1.229002,0.219265,12.737313,0.001714


In [33]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.799607,0.410871,1,5
1,0.940313,0.368447,3,3
2,0.819334,0.814779,3,9
0,0.781974,0.883142,1,1
1,0.042346,0.533572,4,6


In [34]:
# In dataframe statistical tests!
df.select(
    pds.query_ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.819591,0.412469}","{25.195031,0.91122}","{0.548541,0.700106}"


In [35]:
# Can also be done in group by context
df.group_by("market_id").agg(
    pds.query_ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{1.410648,0.158442}","{41.747134,0.235157}","{0.170655,0.953422}"
1,"{-0.832603,0.405128}","{18.694255,0.992338}","{1.567889,0.180314}"
2,"{0.84268,0.399468}","{27.49369,0.844749}","{1.332102,0.255724}"


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [36]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [37]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.447338,0.509968,0.191159,0.76571,4.004095,18
1,0.497393,0.087446,0.040273,0.603727,6.870764,11
2,0.179121,0.545541,0.350251,0.548835,5.518944,17
3,0.077923,0.235837,0.524266,0.38244,8.566175,19
4,0.947276,0.547717,0.074081,0.685048,1.7382,13


In [38]:
df.with_columns(
    pds.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.447338,0.509968,0.191159,0.76571,4.004095,856
1,0.497393,0.087446,0.040273,0.603727,6.870764,235
2,0.179121,0.545541,0.350251,0.548835,5.518944,344
3,0.077923,0.235837,0.524266,0.38244,8.566175,116
4,0.947276,0.547717,0.074081,0.685048,1.7382,330


In [39]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.447338,0.509968,0.191159,0.76571,4.004095,"[0, 634, … 1711]"
1,0.497393,0.087446,0.040273,0.603727,6.870764,"[1, 1599, … 174]"
2,0.179121,0.545541,0.350251,0.548835,5.518944,"[2, 1225, … 139]"
3,0.077923,0.235837,0.524266,0.38244,8.566175,"[3, 159, … 1595]"
4,0.947276,0.547717,0.074081,0.685048,1.7382,"[4, 1235, … 912]"


In [40]:
# Get all neighbors within radius r
# The point itself is always considered a neighbor to itself.
print(df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head())

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 634, … 1488]  ┆ 219                │
│ 1   ┆ [1, 1599, … 1501] ┆ 112                │
│ 2   ┆ [2, 1225, … 922]  ┆ 235                │
│ 3   ┆ [3, 159, … 981]   ┆ 167                │
│ 4   ┆ [4, 1235, … 947]  ┆ 116                │
└─────┴───────────────────┴────────────────────┘


In [41]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.447338,0.509968,0.191159,0.76571,4.004095,"[0, 634, … 1711]","[0.0, 0.002618, … 0.003052]"
1,0.497393,0.087446,0.040273,0.603727,6.870764,"[1, 1599, … 174]","[0.0, 0.000935, … 0.004375]"
2,0.179121,0.545541,0.350251,0.548835,5.518944,"[2, 1225, … 139]","[0.0, 0.004114, … 0.006569]"
3,0.077923,0.235837,0.524266,0.38244,8.566175,"[3, 159, … 1595]","[0.0, 0.000706, … 0.005611]"
4,0.947276,0.547717,0.074081,0.685048,1.7382,"[4, 1235, … 912]","[0.0, 0.001984, … 0.003481]"


In [42]:
# Filter to only points near the given point
df.filter(
    pds.query_within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
0,0.447338,0.509968,0.191159,0.76571,4.004095
2,0.179121,0.545541,0.350251,0.548835,5.518944
9,0.394521,0.642833,0.341052,0.759847,0.328093
10,0.400082,0.434516,0.747893,0.514297,0.943675
11,0.471185,0.67353,0.277491,0.508503,2.663822


In [43]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.query_within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
0,0.447338,0.509968,0.191159,0.76571,4.004095
90,0.485856,0.571541,0.095408,0.117571,4.868403
151,0.447804,0.549862,0.683425,0.429311,4.587895
162,0.492178,0.548071,0.124832,0.442894,9.519109
187,0.568611,0.503028,0.5355,0.702164,3.795168


In [44]:
df.filter(
    pds.query_within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
162,0.492178,0.548071,0.124832,0.442894,9.519109
371,0.461745,0.499197,0.140168,0.193548,9.901649
382,0.551305,0.456784,0.604726,0.27873,8.125213
404,0.477367,0.483348,0.305392,0.910534,9.503475
568,0.494273,0.483281,0.676793,0.452189,6.774915


In [45]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 371, … 209]",130
1,"[1, 791, … 261]",106
2,"[2, 737, … 1116]",127
3,"[3, 159, … 539]",112
4,"[4, 1827, … 786]",97


# Simple Graph Queries

There is limited functionality in the Graph module currently. E.g. Only constant cost per edge.

Graph queries are very expensive.

In [46]:
# friends.select(
#     pl.col("friends").graph.eigen_centrality() # .arg_max()
# ).head()

In [47]:
# Turn friends to a table suitable for graph analytics
df_graph = friends.select(
    pl.col("id"),
    pl.col("friends"),
).explode(pl.col("friends")).with_columns(
    pl.col("id").cast(pl.UInt32),
    pl.col("friends").cast(pl.UInt32),
)
df_graph.head()

id,friends
u32,u32
0,0
0,371
0,1879
0,1462
0,1035


In [48]:
df_graph.select(
    # Shortest path to the node with id = 3
    # Node and link can be str | pl.Expr
    pds.query_shortest_path(node = "id", link = pl.col("friends"), target = 3, cost = None, parallel=True).alias("shortest_path")
).unnest("shortest_path").sort("id")

id,path
u32,list[u32]
0,"[1845, 1193, … 3]"
1,"[261, 757, … 3]"
2,"[1116, 883, 3]"
3,[]
4,"[140, 48, … 3]"
…,…
1995,"[1664, 1114, … 3]"
1996,"[1358, 199, … 3]"
1997,"[1255, 1008, … 3]"


In [49]:
df_graph.select(
    # Almost every node can reach node 3, and the number is the number steps to reach it
    # This is a way faster way to filter results if you don't need the actual path
    pds.query_node_reachable("id", "friends", target = 3).alias("reach")
).unnest("reach")

id,reachable,steps
u32,bool,u32
835,true,3
512,true,6
874,true,3
1318,true,3
1478,true,7
…,…,…
543,true,7
1869,true,6
189,true,4


In [50]:
relationships = pl.DataFrame({
    "id": range(5),
    "connections":[[1,2,3,4], [2,3], [4], [0,1,2], [1]],
    # Small values means closer
    "close-ness":[[0.4, 0.3, 0.2, 0.1], [0.1, 1.0], [0.5], [0.1, 0.1, 0.1], [0.1]]
}).with_columns(
    pl.col("id").cast(pl.UInt32),
    pl.col("connections").list.eval(pl.element().cast(pl.UInt32))
).explode(
    pl.col("connections"), pl.col("close-ness")
)

relationships.head(50)

id,connections,close-ness
u32,u32,f64
0,1,0.4
0,2,0.3
0,3,0.2
0,4,0.1
1,2,0.1
…,…,…
2,4,0.5
3,0,0.1
3,1,0.1


In [51]:
# To go to node at id = 1, node 0 would rather go to 4 first and then 1.
relationships.select(
    pds.query_shortest_path("id", "connections", target = 1, cost = "close-ness").alias("path")
).unnest("path").head()

id,path,cost
u32,list[u32],f64
2,"[4, 1]",0.6
4,[1],0.1
3,[1],0.1
1,[],0.0
0,"[4, 1]",0.2


In [52]:
# In and out deg
relationships.select(
    pds.query_node_deg("id", "connections", directed=True).alias("deg")
).unnest("deg")

node,deg
u32,u32
2,1
4,1
3,3
1,2
0,4


# String Nearest Neighbors

This might be very slow for very large vocab / column.

In [53]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"] 
})

In [54]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 1, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [55]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [56]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        threshold = 4,
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [57]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pds.query_similar_words(
        "a",
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"
